## Data

I wil analyse the Forusquare venue data of different neighbourhoods in Aarhus (Denmark).

### We find the most trendy neighbourhoods in Aarhus - according to Culture Trip

The Culture Trip has chosen the most trendy neighbourhoods in Aarhus, and I will scrape their web-page for the information. But first we will import necessary packages.

In [3]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [4]:
liste = []
url = 'https://theculturetrip.com/europe/denmark/articles/the-most-trendy-neighbourhoods-in-aarhus/'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'lxml')
tags = soup('h2')
for tag in tags:
    if (str(tag.contents[0]).split(' ', 1)[0]) == 'The':
        liste.append('Graven')
    else:   
        liste.append(str(tag.contents[0]))
trend = pd.DataFrame(liste, columns=['Neighbourhoods'])
trend

,Neighbourhoods
0,Graven
1,City centre
2,Frederiksberg
3,Vestergade
4,Aarhus Ø
5,Vesterbro
6,Trøjborg


### We find the location of these trendy neighbourhoods
    

We use the Nominatim geocoder in Geopy.

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
knatt = pd.DataFrame({'Neighbourhood': [], 'lat': [], 'lng': []})
for code in trend.Neighbourhoods:
    try:
        location = geolocator.geocode(code+" aarhus")
        knatt = knatt.append({'Neighbourhood': code, 'lat': location.latitude, 'lng': location.longitude}, ignore_index=True)
    except:
        print(code)
knatt

,Neighbourhood,lat,lng
0,Graven,56.158894,10.209434
1,City centre,56.149628,10.213405
2,Frederiksberg,56.146850,10.194690
3,Vestergade,56.157513,10.201332
4,Aarhus Ø,56.164402,10.224361
5,Vesterbro,56.158210,10.198991
6,Trøjborg,56.172778,10.215807


### We import folium for plotting the neighbourhoods in a map of Aarhus

First we import packages to handle folium.

In [9]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  16.73 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.40 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.15 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  44.69 MB/s
Libraries imported.


### The trendy neighbourhoods shown on map

In [11]:
address = 'Aarhus C'

geolocator = Nominatim(user_agent="aarhus_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Aarhus are {}, {}.'.format(latitude, longitude))

# create map of Aarhus using latitude and longitude values
map_aarhus = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, neighbourhood in zip(knatt['lat'], knatt['lng'], knatt['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_aarhus)  
    
map_aarhus

The geograpical coordinate of Aarhus are 56.1496278, 10.2134046.


### We find the "food" venus in the neighbourhoods

In [14]:
CLIENT_ID = '1APIKEUUY3N0O0CG22IPYPWFVL33NLZPJN4RVSHXBO315QIS' # your Foursquare ID
CLIENT_SECRET = 'Y21WCG50Q5S4VLNYQOFIUQ1IHBSQTDUWLDS0NVQX1QBXIDQU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

food_categories = '4d4b7105d754a06374d81259'
seafood_category = '4bf58dd8d48988d1ce941735' #This is not used for now but mayby later


#define function to get near by venues in each naighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            food_categories,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                    'Neighbourhood Latitude', 
                    'Neighbourhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    
    return(nearby_venues)

# Get the venues to each neighbourhood in Aarhus
aarhus_venues = getNearbyVenues(names=knatt['Neighbourhood'],
                                   latitudes=knatt['lat'],
                                   longitudes=knatt['lng']
                                  )


aarhus_venues.head()

Your credentails:
CLIENT_ID: 1APIKEUUY3N0O0CG22IPYPWFVL33NLZPJN4RVSHXBO315QIS
CLIENT_SECRET:Y21WCG50Q5S4VLNYQOFIUQ1IHBSQTDUWLDS0NVQX1QBXIDQU
Graven
City centre
Frederiksberg
Vestergade
Aarhus Ø
Vesterbro
Trøjborg


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Graven,56.158894,10.209434,Café Drudenfuss,56.158923,10.209439,Café
1,Graven,56.158894,10.209434,Vaca - Mexican Finger Food,56.158121,10.211954,Mexican Restaurant
2,Graven,56.158894,10.209434,Chido Mexican Grill,56.159684,10.212708,Mexican Restaurant
3,Graven,56.158894,10.209434,Langhoff & Juul,56.159547,10.207843,Café
4,Graven,56.158894,10.209434,Nummer 24,56.158794,10.210059,Bakery


### With this data we can make a k-mean clustering analysis in order to recomend a location for the new seafood restaurant